In [1]:
# Step 1: Install and import necessary libraries
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam

In [2]:
# Step 2: Load the dataset
data = pd.read_csv('data.csv')  
data.head()

,Input,Output
0,move north,move north
1,go north,move north
2,walk north,move north
3,head north,move north
4,travel north,move north


In [3]:
# Step 3: Preprocess the data
# Convert the text commands into numerical vectors using Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['Input'])
X = tokenizer.texts_to_sequences(data['Input'])

# Pad sequences to ensure they all have the same length
from tensorflow.keras.preprocessing.sequence import pad_sequences
X = pad_sequences(X, padding='post',maxlen = 5)

# Label encode the output actions
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(data['Output'])

In [4]:
# Step 4: Build the neural network model
model = Sequential()

model.add(Embedding(input_dim=len(tokenizer.word_index) + 1, output_dim=10, input_length=X.shape[1]))
model.add(GlobalAveragePooling1D())
model.add(Dense(6, activation='relu'))
model.add(Dense(len(label_encoder.classes_), activation='softmax'))

# Compile the model
model.compile(optimizer=Adam(), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [5]:
# Step 5: Train the model
history = model.fit(X, y, epochs=1000, batch_size=1024)

Epoch 1/1000
1/1 [==============================] - 1s 862ms/step - loss: 2.8905 - accuracy: 0.0720
Epoch 2/1000
1/1 [==============================] - 0s 5ms/step - loss: 2.8899 - accuracy: 0.0800
Epoch 3/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.8892 - accuracy: 0.0800
Epoch 4/1000
1/1 [==============================] - 0s 5ms/step - loss: 2.8885 - accuracy: 0.0880
Epoch 5/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.8878 - accuracy: 0.0880
Epoch 6/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.8871 - accuracy: 0.0960
Epoch 7/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.8864 - accuracy: 0.0960
Epoch 8/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.8857 - accuracy: 0.0960
Epoch 9/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.8849 - accuracy: 0.0960
Epoch 10/1000
1/1 [==============================] - 0s 4ms/step - loss: 2.8842 - accuracy: 0.0960
Epoch 11/1000
1/1

In [6]:
# Step 6: Evaluate the model
accuracy = model.evaluate(X, y)
print(f"Model accuracy: {accuracy[1]:.4f}")

4/4 [==============================] - 0s 2ms/step - loss: 0.2429 - accuracy: 1.0000
Model accuracy: 1.0000


In [7]:
import pickle

# Save the Tokenizer to a pickle file
with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

# Save the LabelEncoder to a pickle file
with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

#Save the model itself
model.save('model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
# Step 7: Make predictions for test commands
def predict_command(command):
    sequence = tokenizer.texts_to_sequences([command])
    padded_sequence = pad_sequences(sequence, padding='post', maxlen=5)
    prediction = model.predict(padded_sequence)
    predicted_label = np.argmax(prediction, axis=1)
    return label_encoder.inverse_transform(predicted_label)[0]


print(predict_command("head north"))
print(predict_command("move south"))

1/1 [==============================] - 0s 57ms/step
move north
1/1 [==============================] - 0s 12ms/step
move south


In [9]:
import mlflow
import mlflow.keras
import pickle
import os

# Set MLflow server URI 
mlflow.set_tracking_uri("http://192.168.192.25:5600")
mlflow.set_experiment("seq2seq-model")

# Save the model, tokenizer, and label encoder locally first
model.save('command_parser_model.h5')

with open('tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

with open('label_encoder.pkl', 'wb') as f:
    pickle.dump(label_encoder, f)

# Log model and artifacts under a new run
with mlflow.start_run(run_name="v4"):

    # Log the model
    mlflow.keras.log_model(model, "model")

    # Log tokenizer as an artifact
    mlflow.log_artifact('tokenizer.pkl', 'artifacts/tokenizer')

    # Log label encoder as an artifact
    mlflow.log_artifact('label_encoder.pkl', 'artifacts/label_encoder')
    

    # Log metrics from the training history
    for epoch in range(len(history.history['accuracy'])):
        # Log training metrics
        mlflow.log_metric('accuracy', history.history['accuracy'][epoch], step=epoch)
        mlflow.log_metric('loss', history.history['loss'][epoch], step=epoch)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
2024/12/28 20:37:06 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.
2024/12/28 20:37:10 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run v4 at: http://192.168.192.25:5600/#/experiments/274938956705297881/runs/16c8c648ebb24b37b88b20c6bc960ce6
🧪 View experiment at: http://192.168.192.25:5600/#/experiments/274938956705297881
